# 1. Models microservices

In [ ]:
# model_a.py
from fastapi import FastAPI
app = FastAPI()

@app.get("/a/{input_data}")
async def predict(input_data: str):
    return {"model": "A", "input": input_data, "prediction": True}

# model_b.py
from fastapi import FastAPI
app = FastAPI()

@app.get("/b/{input_data}")
async def predict(input_data: str):
    return {"model": "B", "input": input_data, "prediction": False}

# model_c.py
from fastapi import FastAPI, HTTPException
import random
app = FastAPI()

@app.get("/c/{input_data}")
async def predict(input_data: str):
    if random.choice([True, False]):
        return {"model": "C", "input": input_data, "prediction": True}
    else:
        raise HTTPException(status_code=400, detail="Model C failed to predict")


# 2. Client with retries

In [ ]:
# client.py
import httpx
import asyncio

async def call_model(model: str, input_data: str):
    port_map = {"a": 8001, "b": 8002, "c": 8003}
    url = f"http://localhost:{port_map[model]}/{model}/{input_data}"
    async with httpx.AsyncClient() as client:
        for attempt in range(3):  # Retry up to 3 times
            try:
                response = await client.get(url)
                response.raise_for_status()
                return response.json()
            except (httpx.HTTPError, httpx.RequestError) as e:
                print(f"Attempt {attempt + 1} failed for model {model}: {e}")
                await asyncio.sleep(1)
        raise Exception(f"Failed to get successful response from model {model} after 3 attempts.")


# 3. Model Server & Supervisor (Routing API)

In [ ]:
# model_supervisor.py
from fastapi import FastAPI, HTTPException
import asyncio
from client import call_model

app = FastAPI()

@app.get("/api/predict/{input_data}")
async def predict(input_data: str):
    h = hash(input_data) % 10
    if h < 4:
        model = "a"
    elif h < 6:
        model = "b"
    else:
        model = "c"
    try:
        result = await call_model(model, input_data)
        return result
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
 ##
